<a href="https://colab.research.google.com/gist/thomngt/3d402656c228d67d279997c7ad5548c9/bigdata-_project_amazon_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
0.## Kết nối Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
folder = '/content/gdrive/My Drive/Colab Notebooks/Bigdata'

Mounted at /content/gdrive


#THƯ VIỆN CẦN THIẾT PHỤC VỤ CHO BÀI TOÁN



In [ ]:
#Thư viện cần thiết:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
import tensorflow as tf
# Tắt những cảnh báo
import warnings
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes as nb
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, Embedding,  BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# PHẦN 1: TỔNG QUAN DỮ LIỆU

In [ ]:
data=pd.read_csv(folder +'/amazon_reviews.csv',encoding='ISO-8859-1')
data.head()

,Name,Rating,Title,Date,Review_Text
0,Brdove,5.0,5.0 out of 5 stars\nVery roomy and comfortable...,05/02/2024,I like that these have extra room in the toes ...
1,Shariante Hawes,5.0,5.0 out of 5 stars\nComfortable and great qual...,28/02/2024,"They are very durable, I've had them about a m..."
2,Ebie,4.0,4.0 out of 5 stars\nExactly as described,27/02/2024,"Great shoes, do exactly what they're supposed ..."
3,DizNerd,1.0,1.0 out of 5 stars\nNot so slip resistant,10/10/2023,"Heavy and bulky feeling, but the foot bed is n..."
4,kim,5.0,5.0 out of 5 stars\nSturdy shoes,23/02/2024,Comfortable very sturdy


Thông tin cơ bản về file data

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         1400 non-null   object 
 1   Rating       1400 non-null   float64
 2   Title        1400 non-null   object 
 3   Date         1400 non-null   object 
 4   Review_Text  1400 non-null   object 
dtypes: float64(1), object(4)
memory usage: 54.8+ KB


#PHẦN 2: XÂY DỰNG MÔ HÌNH

###Tiền xử lý dữ liệu

In [ ]:
data.head()

,Name,Rating,Title,Date,Review_Text
0,Brdove,5.0,5.0 out of 5 stars\nVery roomy and comfortable...,05/02/2024,I like that these have extra room in the toes ...
1,Shariante Hawes,5.0,5.0 out of 5 stars\nComfortable and great qual...,28/02/2024,"They are very durable, I've had them about a m..."
2,Ebie,4.0,4.0 out of 5 stars\nExactly as described,27/02/2024,"Great shoes, do exactly what they're supposed ..."
3,DizNerd,1.0,1.0 out of 5 stars\nNot so slip resistant,10/10/2023,"Heavy and bulky feeling, but the foot bed is n..."
4,kim,5.0,5.0 out of 5 stars\nSturdy shoes,23/02/2024,Comfortable very sturdy


In [ ]:
#data trước khi tiền xử lý
df_main_class =data[["Review_Text","Rating"]]
df_main_class

,Review_Text,Rating
0,I like that these have extra room in the toes ...,5.0
1,"They are very durable, I've had them about a m...",5.0
2,"Great shoes, do exactly what they're supposed ...",4.0
3,"Heavy and bulky feeling, but the foot bed is n...",1.0
4,Comfortable very sturdy,5.0
...,...,...
1395,It was really too narrow at the heel. I used t...,3.0
1396,"According to my wife, these are the best. 16,...",5.0
1397,They are very comfortable. Support my very hi...,5.0
1398,Nonslip is great. And a fairly comfortable sho...,4.0


#####Tiền xử lý dữ liệu

Gắn nhãn cho cột Rating

In [ ]:
rating = data["Rating"]
def label_func(rating):
    if rating >=4:
        return "Positive"
    elif rating == 3:
            return "Neutral"
    else:
        return "Negative"

data["Sentiment"] = data["Rating"].apply(lambda x: label_func(x))

In [ ]:
#Tạo hàm tiền xử lý dữ liệu
stop_words = set(stopwords.words('english'))

def clean_reviews(text):
    # Xóa html
    regex = re.compile('<.*?>')
    text = re.sub(regex, '', text)

    # Xóa những ký tự đặc biệt
    pattern = re.compile('[^a-zA-z0-9\s]')
    text = re.sub(pattern, '', text)

    # Xóa chữ số
    pattern = re.compile('\d+')
    text = re.sub(pattern, '', text)

    # Chuyển đổi text sang chữ thường
    text = text.lower()

    # Tokenization of words
    text = word_tokenize(text)

    # Lemmatization of words
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]

    # Stop words removal
    text = [word for word in text if word not in stop_words]

    # Nối các từ tạo thành một chuỗi
    text = ' '.join(text)

    return text
data['Review_Text'] = data['Review_Text'].apply(clean_reviews)

In [ ]:
#data sau khi tiền xử lý
df_main_class = data[["Review_Text","Sentiment"]]
df_main_class

,Review_Text,Sentiment
0,like extra room toe foot cramped also material...,Positive
1,durable ive month easy clean comfortable good ...,Positive
2,great shoe exactly theyre supposed perfect res...,Positive
3,heavy bulky feeling foot bed nice supportivei ...,Negative
4,comfortable sturdy,Positive
...,...,...
1395,wa really narrow heel used buy lot skechers so...,Neutral
1396,according wife best step day kitchen must right,Positive
1397,comfortable support high arch perfectly foot s...,Positive
1398,nonslip great fairly comfortable shoe hour foo...,Positive


#### Tạo tập dữ liệu kiểm thử và kiểm tra

In [ ]:
X = df_main_class['Review_Text']
y = df_main_class['Sentiment']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 80-20
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, stratify= y, random_state=42)

print('Tập Train (80%): ', train_x.shape)
print('Tập Test (20%: ', test_x.shape)

Tập Train (80%):  (1120,)
Tập Test (20%:  (280,)


###Xây dựng mô hình

##### Naive Bayes Model:


######Trích chọn đặc trưng theo phương pháp TF_IDF

In [ ]:
#Trích chọn đặc trưng theo phương pháp TF_IDF
vector =TfidfVectorizer(analyzer ='word',
                        max_features=35000,
                        stop_words = 'english')
vector.fit(df_main_class['Review_Text'])
xtrain_tfidf = vector.transform(train_x)
xtest_tfidf = vector.transform(test_x)
print('Kết quả vector hóa tập Train sang dạng số:')
print(xtrain_tfidf.data)
print(xtrain_tfidf.shape)
print('Kết quả vector hóa tập Test sang dạng số:')
print(xtest_tfidf.data)
print(xtest_tfidf.shape)

Kết quả vector hóa tập Train sang dạng số:
[0.11583178 0.06730315 0.11583178 ... 0.17385179 0.88958897 0.45676194]
(1120, 186)
Kết quả vector hóa tập Test sang dạng số:
[0.42995866 0.42995866 0.24982413 ... 0.28727231 0.42995866 0.42995866]
(280, 186)


######Trích chọn đặc trưng theo phương pháp BoW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Tạo một CountVectorizer
vectorizer = CountVectorizer(analyzer='word', max_features=20000, stop_words='english')

# Khớp dữ liệu văn bản vào vectorizer để tạo BoW
vectorizer.fit(df_main_class['Review_Text'])

# Chuyển đổi dữ liệu huấn luyện và kiểm tra thành dạng ma trận BoW
xtrain_bow = vectorizer.transform(train_x)
xtest_bow = vectorizer.transform(test_x)

# In ra thông tin về ma trận BoW
print('Kết quả vector hóa tập Train sang dạng số:')
print(xtrain_bow.toarray())
print(xtrain_bow.shape)
print('Kết quả vector hóa tập Test sang dạng số:')
print(xtest_bow.toarray())
print(xtest_bow.shape)


Kết quả vector hóa tập Train sang dạng số:
[[1 0 1 ... 0 1 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
(1120, 186)
Kết quả vector hóa tập Test sang dạng số:
[[0 1 0 ... 1 0 0]
 [0 1 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 1 0 0]]
(280, 186)


In [ ]:
#Sử dụng mô hình Naive Bayes với TF-IDF
#Sử dụng thuật toán MultinomialNB
MultiNB = nb.MultinomialNB(alpha=0.1, fit_prior=True)
MultiNB.fit(xtrain_tfidf, train_y)


MultinomialNB(alpha=0.1)

In [ ]:
# Huấn luyện mô hình với tập Train
MultiNB.fit(xtrain_tfidf, train_y)
train_score = MultiNB.score(xtrain_tfidf, train_y)
print(f'Độ chính xác của mô hình trên tập Train: {round(train_score*100,2)}%')

# Tính thời gian huấn luyện trên tập Train
start_time_train = time.time()
MultiNB.fit(xtrain_tfidf, train_y)
end_time_train = time.time()
train_time = end_time_train - start_time_train
print(f'Thời gian huấn luyện trên tập Train: {train_time:.4f} giây')

#Sử dụng mô hình huấn luyện dự đoán trên tập Test
y_pred = MultiNB.predict(xtest_tfidf)
#Đánh giá mô hình trên tập Test
print('classification_report: \n', classification_report(test_y, y_pred))
# Tính thời gian dự đoán trên tập Test
start_time_test = time.time()
y_pred = MultiNB.predict(xtest_tfidf)
end_time_test = time.time()
test_time = end_time_test - start_time_test
print(f'Thời gian dự đoán trên tập Test: {test_time:.4f} giây')

Độ chính xác của mô hình trên tập Train: 100.0%
Thời gian huấn luyện trên tập Train: 0.0237 giây
classification_report: 
               precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        28
     Neutral       1.00      1.00      1.00        28
    Positive       1.00      1.00      1.00       224

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280

Thời gian dự đoán trên tập Test: 0.0008 giây


In [ ]:
#Sử dụng mô hình Naive Bayes với BoW
from sklearn import naive_bayes as nb

#Sử dụng thuật toán MultinomialNB
MultiNB = nb.MultinomialNB(alpha = 0.1, fit_prior=True)
MultiNB.fit(xtrain_bow, train_y)


MultinomialNB(alpha=0.1)

In [ ]:
from sklearn.metrics import accuracy_score
# Tính thời gian huấn luyện trên tập Train
start_time_train = time.time()
MultiNB.fit(xtrain_bow, train_y)
end_time_train = time.time()
train_time = end_time_train - start_time_train
print(f'Thời gian huấn luyện trên tập Train: {train_time:.4f} giây')
#Sử dụng mô hình huấn luyện dự đoán trên tập Test
y_pred = MultiNB.predict(xtest_bow)
#Đánh giá mô hình trên tập Test
print('classification_report: \n', classification_report(test_y, y_pred))
# Tính thời gian dự đoán trên tập Test
start_time_test = time.time()
y_pred = MultiNB.predict(xtest_bow)
end_time_test = time.time()
test_time = end_time_test - start_time_test
print(f'Thời gian dự đoán trên tập Test: {test_time:.4f} giây')

Thời gian huấn luyện trên tập Train: 0.0166 giây
classification_report: 
               precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        28
     Neutral       1.00      1.00      1.00        28
    Positive       1.00      1.00      1.00       224

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280

Thời gian dự đoán trên tập Test: 0.0007 giây


#####MaxentClassifier(Logistic Regression)

In [ ]:
LogReg = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=100)
LogReg.fit(xtrain_tfidf, train_y)

LogisticRegression()

In [ ]:
# Huấn luyện mô hình với tập Train
LogReg.fit(xtrain_tfidf, train_y)
train_score = LogReg.score(xtrain_tfidf, train_y)
print(f'Độ chính xác của mô hình trên tập Train: {round(train_score*100,2)}%')

# Tính thời gian huấn luyện trên tập Train
start_time_train = time.time()
LogReg.fit(xtrain_tfidf, train_y)
end_time_train = time.time()
train_time = end_time_train - start_time_train
print(f'Thời gian huấn luyện trên tập Train: {train_time:.4f} giây')

#Sử dụng mô hình huấn luyện dự đoán trên tập Test
y_pred_tfidf = LogReg.predict(xtest_tfidf)
#Đánh giá mô hình trên tập Test
print('classification_report: \n', classification_report(test_y, y_pred_tfidf))
# Tính thời gian dự đoán trên tập Test
start_time_test = time.time()
y_pred_tfidf = LogReg.predict(xtest_tfidf)
end_time_test = time.time()
test_time = end_time_test - start_time_test
print(f'Thời gian dự đoán trên tập Test: {test_time:.4f} giây')

Độ chính xác của mô hình trên tập Train: 100.0%
Thời gian huấn luyện trên tập Train: 0.0804 giây
classification_report: 
               precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        28
     Neutral       1.00      1.00      1.00        28
    Positive       1.00      1.00      1.00       224

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280

Thời gian dự đoán trên tập Test: 0.0153 giây


In [ ]:
LogReg = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=100)
LogReg.fit(xtrain_bow, train_y)

LogisticRegression()

In [ ]:
# Huấn luyện mô hình với tập Train
LogReg.fit(xtrain_bow, train_y)
train_score = LogReg.score(xtrain_bow, train_y)
print(f'Độ chính xác của mô hình trên tập Train: {round(train_score*100,2)}%')

# Tính thời gian huấn luyện trên tập Train
start_time_train = time.time()
LogReg.fit(xtrain_bow, train_y)
end_time_train = time.time()
train_time = end_time_train - start_time_train
print(f'Thời gian huấn luyện trên tập Train: {train_time:.4f} giây')

#Sử dụng mô hình huấn luyện dự đoán trên tập Test
y_pred_bow = LogReg.predict(xtest_bow)
#Đánh giá mô hình trên tập Test
print('classification_report: \n', classification_report(test_y, y_pred_bow))
# Tính thời gian dự đoán trên tập Test
start_time_test = time.time()
y_pred_bow = LogReg.predict(xtest_bow)
end_time_test = time.time()
test_time = end_time_test - start_time_test
print(f'Thời gian dự đoán trên tập Test: {test_time:.4f} giây')

Độ chính xác của mô hình trên tập Train: 100.0%
Thời gian huấn luyện trên tập Train: 0.0357 giây
classification_report: 
               precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        28
     Neutral       1.00      1.00      1.00        28
    Positive       1.00      1.00      1.00       224

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280

Thời gian dự đoán trên tập Test: 0.0004 giây


#####Deep Learning (RNN)


In [ ]:
print(df_main_class.columns)

Index(['Review_Text', 'Sentiment'], dtype='object')


In [ ]:
from keras.preprocessing.sequence import pad_sequences


In [ ]:
tokenizer = Tokenizer()
# chuyển đổi reviews thành list
reviews_to_list = df_main_class['Review_Text'].tolist()
tokenizer.fit_on_texts(reviews_to_list)
# Tạo chuỗi văn bản
text_sequences = tokenizer.texts_to_sequences(reviews_to_list)
# one hot encoding
df_main_class = pd.get_dummies(df_main_class, columns = ['Sentiment'])

# setting maximum words
max_words = 500

# X (input) to the model
# using pad_sequences and y (output)
X = pad_sequences(text_sequences, maxlen = max_words)
y = df_main_class[['Sentiment_Negative','Sentiment_Neutral','Sentiment_Positive']]
print(X.shape, y.shape)

(1400, 500) (1400, 3)


In [ ]:
# Creating a RNN model
rnn = Sequential(name="Simple_RNN")
rnn.add(Embedding(len(tokenizer.word_index)+1,
                        max_words,
                        input_length=max_words))

rnn.add(SimpleRNN(128,activation='relu',return_sequences=True))

rnn.add(SimpleRNN(64,activation='relu',return_sequences=False))

rnn.add(Dense(3, activation='softmax'))

# printing model summary
print(rnn.summary())

Model: "Simple_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 500)          108000    
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 500, 128)          80512     
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 201059 (785.39 KB)
Trainable params: 201059 (785.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 80-20
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, stratify= y, random_state=42)

print('Tập Train (80%): ', train_x.shape)
print('Tập Test (20%: ', test_x.shape)

Tập Train (80%):  (1120, 500)
Tập Test (20%:  (280, 500)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Compiling model
rnn.compile(
	loss="categorical_crossentropy",
	optimizer='adam',
	metrics=['accuracy']
)

#Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# Training the model
history = rnn.fit(train_x, train_y,
						batch_size=64,
						epochs=50,
						verbose=1,
						validation_data = (test_x, test_y),
						callbacks=[early_stopping])

print("Simple_RNN Score---> ", rnn.evaluate(test_x, test_y, verbose=1))


Epoch 1/50
18/18 [==============================] - 22s 1s/step - loss: 0.4176 - accuracy: 0.8857 - val_loss: 0.0449 - val_accuracy: 1.0000
Epoch 2/50
18/18 [==============================] - 24s 1s/step - loss: 0.0129 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 3/50
18/18 [==============================] - 33s 2s/step - loss: 5.2917e-04 - accuracy: 1.0000 - val_loss: 2.4170e-04 - val_accuracy: 1.0000
Epoch 4/50
18/18 [==============================] - 28s 2s/step - loss: 6.9830e-05 - accuracy: 1.0000 - val_loss: 2.1190e-05 - val_accuracy: 1.0000
Epoch 5/50
18/18 [==============================] - 23s 1s/step - loss: 1.3142e-05 - accuracy: 1.0000 - val_loss: 8.8051e-06 - val_accuracy: 1.0000
Epoch 6/50
18/18 [==============================] - 19s 1s/step - loss: 7.7999e-06 - accuracy: 1.0000 - val_loss: 6.8804e-06 - val_accuracy: 1.0000
Epoch 7/50
18/18 [==============================] - 21s 1s/step - loss: 6.3043e-06 - accuracy: 1.0000 - val_loss: 5.7598e-06 - v

In [ ]:
y_true = np.argmax(test_y.values, axis=1)
y_true.shape
y_pred = np.argmax(rnn.predict(test_x), axis=1)
report = classification_report(y_true, y_pred)
print(report)


9/9 [==============================] - 3s 272ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        28
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00       224

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280

